<a href="https://colab.research.google.com/github/ADI201998/sparse-to-depth-GAN/blob/master/depth_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
!pip install tensorflow-gpu

In [0]:
import cv2
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
import os
import h5py
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv2D, UpSampling2D, Conv2DTranspose
from keras.layers import ZeroPadding2D, Activation, BatchNormalization, MaxPooling2D, Dense, Add, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Input, Dropout
from keras import losses
from keras.optimizers import Adam
from keras.utils import to_categorical

input_shape_generator = (480, 640, 4)
input_shape_discriminator = (480, 640, 1)

Using TensorFlow backend.


In [0]:
def dense_to_sparse(rgb, depth):
        """
        Samples pixels with `num_samples`/#pixels probability in `depth`.
        Only pixels with a maximum depth of `max_depth` are considered.
        If no `max_depth` is given, samples in all pixels
        """
        # max depth is np.inf
        mask_keep = depth > 0
        if np.inf is not np.inf:
                mask_keep = np.bitwise_and(mask_keep, depth <= 1.0)
        n_keep = np.count_nonzero(mask_keep)
        if n_keep == 0:
                return mask_keep
        else:
                prob = float(200) / n_keep
                return np.bitwise_and(mask_keep, np.random.uniform(0, 1, depth.shape) < prob)

In [0]:
def create_sparse_depth(rgb, depth):
        mask_keep = dense_to_sparse(rgb, depth)
        sparse_depth = np.zeros(depth.shape)
        sparse_depth[mask_keep] = depth[mask_keep]
        return sparse_depth

In [0]:
def create_rgbd(rgb, depth):
        sparse_depth = create_sparse_depth(rgb, depth)
        rgbd = np.append(rgb, np.expand_dims(sparse_depth, axis=2), axis=2)
        rgbd = rgbd[np.newaxis, :, :, :]
        return rgbd

In [0]:
def dataset():
        X_train = np.empty((1400, 480, 640, 4))
        Y_train = np.empty((1400, 480, 640, 1))
        X_test = np.empty((280, 480, 640, 4))
        Y_test = np.empty((280, 480, 640, 1))
        rootdir_train = '/content/drive/My Drive/Dataset/train/'
        rootdir_val = '/content/drive/My Drive/Dataset/val/official/'
        print("TRAIN")
        j = i = 0
        for subdir, dirs, files in sorted(os.walk(rootdir_train)):
                print(j)
                files = sorted(files)
                print(len(files))
                if len(files) != 0:
                        for k in range(5):
                              #print(i)
                              path = subdir + '/' + files[k]
                              h5f = h5py.File(path, "r")
                              rgb = np.array(h5f['rgb'])
                              rgb = np.transpose(rgb, (1, 2, 0))
                              depth = np.array(h5f['depth'])
                              depth  = depth /9
                              #cv2.imshow("rgb", rgb)
                              #cv2.imshow("depth", depth)
                              X_train[i] = create_rgbd(rgb, depth)
                              Y_train[i] = depth[:, :, np.newaxis]
                              i = i+1
                              #cv2.waitKey(0)
                              #cv2.destroyAllWindows()
                        j = j+1
                                                
        print("TEST")
        for subdir, dirs, files in sorted(os.walk(rootdir_val)):
                files = sorted(files)
                print(len(files))
                for i in range(280):
                        print(i)
                        path = subdir + '/' + files[i]
                        h5f = h5py.File(path, "r")
                        rgb = np.array(h5f['rgb'])
                        rgb = np.transpose(rgb, (1, 2, 0))
                        depth = np.array(h5f['depth'])
                        depth  = depth /9
                        X_test[i] = create_rgbd(rgb, depth)
                        Y_test[i] = depth[:, :, np.newaxis]
        print("OVER")
        print("Done")
        return X_train, Y_train, X_test, Y_test

In [0]:
def generator_model():
    #encoder
    X_input = Input(input_shape_generator)
    X = Conv2D(64, 3, activation = 'relu', padding='same', kernel_initializer = 'he_normal')(X_input)
    X = BatchNormalization(axis=3)(X)
    X = MaxPooling2D((2,2))(X)
    X = Conv2D(128, 3, activation = 'relu', padding='same', kernel_initializer = 'he_normal')(X)
    X = BatchNormalization(axis=3)(X)
    X = MaxPooling2D((2,2))(X)
    X = Conv2D(256, 3,activation = 'relu', padding='same', kernel_initializer = 'he_normal')(X)
    X = BatchNormalization(axis=3)(X)
    X = MaxPooling2D((2,2))(X)
    X = Conv2D(256, 1, activation = 'relu', padding='same', kernel_initializer = 'he_normal')(X)
    X = BatchNormalization(axis=3)(X)
    X1 = X
    X = Conv2D(512, 3, activation = 'relu', padding='same', kernel_initializer = 'he_normal')(X)
    X = BatchNormalization(axis=3)(X)
    X = MaxPooling2D((2,2))(X)
    X = Conv2D(512, 1, activation = 'relu', padding='same', kernel_initializer = 'he_normal')(X)
    X = BatchNormalization(axis=3)(X)
    X2 = X
    X = Conv2D(512, 3, activation = 'relu', padding='same', kernel_initializer = 'he_normal')(X)
    X = BatchNormalization(axis=3)(X)
    X = MaxPooling2D((2,2))(X)
    X3 = X 
    X = Conv2D(512, 3, activation = 'relu', padding='same', kernel_initializer = 'he_normal')(X)
    X = BatchNormalization(axis=3)(X)
    X = MaxPooling2D((1,2))(X)

    #decoder
    X = Conv2D(512, 3, activation = 'relu', padding='same', kernel_initializer = 'he_normal')(UpSampling2D((1,2))(X))
    X = BatchNormalization(axis=3)(X)
    X = Add()([X,X3])
    X = Dropout(0.5)(X)

    X = Conv2D(512, 3, activation = 'relu', padding='same', kernel_initializer = 'he_normal')(UpSampling2D((2,2))(X))
    X = BatchNormalization(axis=3)(X)
    X = Add()([X,X2])

    X = Conv2D(256, 3, activation = 'relu', padding='same', kernel_initializer = 'he_normal')(UpSampling2D((2,2))(X))
    X = BatchNormalization(axis=3)(X)
    X = Add()([X,X1])
    X = Dropout(0.5)(X)

    X = Conv2D(128, 3, activation = 'relu', padding='same', kernel_initializer = 'he_normal')(UpSampling2D((2,2))(X))
    X = BatchNormalization(axis=3)(X)
    X = Conv2D(64, 3, activation = 'relu', padding='same', kernel_initializer = 'he_normal')(UpSampling2D((2,2))(X))
    X = BatchNormalization(axis=3)(X)
    X = Dropout(0.5)(X)
    X = Conv2D(1, 3, activation = 'relu', padding='same', kernel_initializer = 'he_normal')(UpSampling2D((2,2))(X))
    X_out = BatchNormalization(axis=3)(X)

    model = Model(input = X_input, output = X_out, name='Generator')
    return model

In [0]:
 def discriminator_model():
    X_input = Input(input_shape_discriminator)
    X = Conv2D(64, kernel_size=(3,3), activation = 'relu', padding='same', kernel_initializer = 'he_normal')(X_input)
    X = BatchNormalization(axis=3)(X)
    X = MaxPooling2D((2,2))(X)
    X = Conv2D(128, kernel_size=(3,3), activation = 'relu', padding='same', kernel_initializer = 'he_normal')(X)
    X = BatchNormalization(axis=3)(X)
    X = MaxPooling2D((2,2))(X)
    X = Conv2D(256, kernel_size=(3,3), activation = 'relu', padding='same', kernel_initializer = 'he_normal')(X)
    X = BatchNormalization(axis=3)(X)
    X = MaxPooling2D((2,2))(X)
    X = Conv2D(512, kernel_size=(3,3), activation = 'relu', padding='same', kernel_initializer = 'he_normal')(X)
    X = BatchNormalization(axis=3)(X)
    X = MaxPooling2D((2,2))(X)
    X = Conv2D(1024, kernel_size=(3,3), activation = 'relu', padding='same', kernel_initializer = 'he_normal')(X)
    X = BatchNormalization(axis=3)(X)
    X = MaxPooling2D((2,2))(X)
    X = Flatten()(X)
    X_out = Dense(1, activation = 'sigmoid')(X)

    model = Model(input =  X_input, output = X_out, name='Discriminator')
    return model

In [0]:
def gan_model(generator, discriminator):
    gan_input = Input(input_shape_generator)
    x = generator(gan_input)
    gan_output = discriminator(x)

    gan = Model(input = gan_input, output = [x, gan_output], name='GAN')
    return gan

In [0]:
batch_size = 10
epochs = 140
input_shape = (480, 640, 4)
# 280 x 20 
##  INPUT  ##
print("----------------------GETTING INPUTS-----------------------")
X_train, Y_train, X_test, Y_test = dataset()
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

## INITIALIZE MODELS  ##
print("----------------------INITIALIZING MODELS-----------------------")
generator = generator_model()       
discriminator = discriminator_model()
gan = gan_model(generator, discriminator)

## COMPILE MODELS ##
print("----------------------COMPILING MODELS-----------------------")
discriminator.trainable = True
discriminator.compile(optimizer = Adam(4e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
discriminator.trainable = False
gan.compile(optimizer = Adam(4e-4), loss = ['mae', 'binary_crossentropy'], loss_weights=[1, 0.001])
discriminator.trainable = True

print("----------------------STARTING TRAINING-----------------------")

for epoch in range(epochs):
    # SELECTING RANDOM BATCH OF IMAGES                      
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]
    op_imgs = Y_train[idx]

    # GENERATE NEW IMAGES
    generated_img = generator.predict(imgs)

    # TRAIN DISCRIMINATOR
    d_loss_real = discriminator.train_on_batch(op_imgs, valid)
    d_loss_fake = discriminator.train_on_batch(generated_img, fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    discriminator.trainable = False
    # TRAIN GENERATOR
    g_loss = gan.train_on_batch(imgs, [op_imgs, valid])                           #fill 
    
    print("Epochs : ",epoch," D_loss : ", d_loss, " G_loss : ", g_loss)
    discriminator.trainable = True

print("----------------------TRAINING ENDS-----------------------")
generator.save('generator_model.h5')
discriminator.save('disriminator_model.h5')
predicted_imgs = generator.predict(X_test)
for i in range(20):
    #rgb = np.squeeze(X_test[i], axis=0)
    rgb = X_test[i, :, :, 0:3]
    print(rgb.shape)
    depth = Y_test[i, :, :, :]
    print(depth.shape)
    depth_pred = predicted_imgs[i, :, :, :]
    print(depth_pred.shape)
    plt.imshow(rgb)
    plt.show()
    #cv2_imshow(str(i)+"rgb", rgb)
    #depth = np.squeeze(Y_test[i], axis=0)
    
    #cv2_imshow(str(i)+"depth", depth)
    plt.imshow(depth)
    plt.show()
    #depth_pred = np.squeeze(generated_img[i], axis=0)
    #cv2_imshow(str(i)+"depth_pred", depth_pred)
    plt.imshow(depth_pred)
    plt.show()
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()